<a href="https://colab.research.google.com/github/amarquand/PCNtoolkit/blob/dev/notebooks/pcntk_colab_dev_env.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Install condacolab, which will enable us to install using conda
## Outcomment this cell after the kernel has automatically restarted
import os
os.environ['PYTHONPATH'] = ""  # Need to clear the pythonpath to avoid some errors
!echo "pythonpath: $PYTHONPATH"
!pip install -q condacolab
import condacolab
condacolab.install()

pythonpath: 
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...


In [ ]:
# Check if condacolab works
import condacolab
condacolab.check()

# Install libraries with optimized dependencies (BLAS and Rust) using conda
!conda install pymc numba nutpie -c conda-forge

# Install CPU only verion of torch using conda (go to https://pytorch.org/get-started/locally/ if you need another version for your platform)
!conda install pytorch torchvision torchaudio cpuonly -c pytorch

# # Install the pcntoolkit
!pip install https://github.com/amarquand/PCNtoolkit/archive/dev.zip

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - numba
    - nutpie
    - pymc


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |       2_kmp_llvm           6 KB  conda-forge
    arviz-0.20.0               |     pyhd8ed1ab_0         1.4 MB  conda-forge
    atk-1.0-2.38.0             |       h04ea711_2         348 KB  conda-forge
    aws-c-auth-0.7.22          |       h96bc93b_2         103 KB  conda-forge
    aws-c-cal-0.6.14           |       h88a6e22_1          46 KB  conda-forge
    aws-c-common-0.9.19        | 

In [ ]:
import pcntoolkit as ptk

### If you get an error about a cffi version mismatch, bright the site-packages version of cffi up to the dist-packages version using the following line
# !pip install --upgrade cffi==1.17.1 # works if dist-packages version is 1.17.1

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
